# Modelo para predicción de gastos futuros:

Se busca que sea adaptable a la informaciónque suben los usuariosm por que se requiere mecanismos que permitan actualizar el modelo a medida que vaya obteniendo más datos(aprendiaje en línea o incremental) y diseñar una estructura donde los datos históricos del usuario se almacenen de forma que puedan alimentar continuamente el modelo para mejorar las predicciones.

In [1]:
pip install statsmodels pandas fastapi uvicorn joblib


## Series Temporales con ARIMA
Si los datos de los gastos tienen una estructura temporal (mes a mes), podriamos utiliar un modelo como ARIMA, que es útil para hacer predicciones basadas en datos secuenciales.

In [2]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import joblib

In [3]:
# Simulación de datos de ingresos y gastos para 24 meses (datos de ejemplo)
np.random.seed(42)
meses = np.arange(1, 25)
ingresos = np.random.uniform(2000, 5000, size=24)
gastos = ingresos * np.random.uniform(0.6, 0.9, size=24)

In [4]:
# Crear un DataFrame con los datos simulados
data = pd.DataFrame({
    'Mes': meses,
    'Ingresos': ingresos,
    'Gastos': gastos
})

In [5]:
# Solo necesitamos la serie de gastos para ARIMA
gastos_series = data['Gastos']

In [6]:
# Crear el modelo ARIMA
modelo_arima = ARIMA(gastos_series, order=(1, 1, 1))
modelo_arima_fit = modelo_arima.fit()

In [7]:
# Guardar el modelo entrenado para uso futuro
joblib.dump(modelo_arima_fit, 'modelo_arima.pkl')
print("Modelo ARIMA entrenado y guardado.")

Modelo ARIMA entrenado y guardado.


## Ejecución con FastApi

In [8]:
!pip install fastapi uvicorn nest_asyncio joblib


In [9]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

In [10]:
# Solucionar problemas de asincronía en Jupyter
nest_asyncio.apply()

In [11]:
# Crear la aplicación FastAPI
app = FastAPI()

In [12]:
# Cargar el modelo ARIMA entrenado
modelo_arima = joblib.load('modelo_arima.pkl')

In [13]:
# Modelo de entrada para la predicción
class PrediccionEntrada(BaseModel):
    pasos: int  # Número de pasos (meses) para predecir a futuro

In [14]:
# Ruta para realizar predicciones de gastos futuros
@app.post("/predecir-gastos/")
def predecir_gastos(datos: PrediccionEntrada):
    # Realizar la predicción para los próximos N pasos (meses)
    prediccion = modelo_arima.forecast(steps=datos.pasos)
    return {"gastos_futuros": list(prediccion)}

In [15]:
# Ejecutar Uvicorn dentro del notebook
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [25816]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62707 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62707 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:62773 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:62773 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:62871 - "POST /predecir-gastos/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:62885 - "POST /predecir-gastos/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63130 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63130 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:63250 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63250 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25816]
